In [ ]:
import sys
import os
fname = r'C:\Users\agmontesb\Documents\GitHub\excel\tests\test_base_workbook.py'
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(fname), '..')))
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(fname), r'..\tests')))

In [ ]:
import pytest
import openpyxl as px
import pandas as pd
import itertools
import re
import inspect
from typing import Any, Literal, Optional

In [ ]:

from excel_workbook import (
    ExcelWorkbook, ExcelTable, 
    cell_address, cell_pattern, 
    tbl_address, rgn_pattern,
    XlErrors, TABLE_DATA_MAP, EMPTY_CELL, CIRCULAR_REF,
    tbl_pattern, rgn_pattern, flatten_sets
    )

from tests.utilities import TableComparator
import xlfunctions as xlf


In [ ]:
from tests.fixtures import static_workbook as base_workbook

In [ ]:
wb = inspect.unwrap(base_workbook)()
ws = wb.sheets[1]
tbl = ws.tables[1]
df = tbl.data

In [ ]:
tbl

In [ ]:
tbl.get_formula('E12:H17')

In [ ]:
class XlValue:
    def __new__(cls, value):
        if isinstance(value, XlErrors):
            return value
        instanve = super().__new__(cls)
        instanve.value = value
        return instanve

    def __init__(self, value):
        self.value = value

    @staticmethod
    def xl_val_dec(func):
        def inner(self, other_in):
            other = other_in.value if isinstance(other_in, XlValue) else other_in
            cls = (type(1), type(1.0)) if isinstance(self.value, (type(1), type(1.0))) else type(self.value)
            if isinstance(other, cls):
                return func(self, other)
            return XlErrors.VALUE_ERROR
        return inner

    @xl_val_dec
    def __add__(self, other):
        return self.value + other

    @xl_val_dec
    def __radd__(self, other):
        return self.__add__(other)  # Delegate to __add__

    @xl_val_dec
    def __sub__(self, other):
        return self.value - other

    @xl_val_dec
    def __rsub__(self, other):
        return -self.__sub__(other)

    @xl_val_dec
    def __mul__(self, other):
        return self.value * other

    @xl_val_dec
    def __rmul__(self, other):
        return self.__mul__(other)

    @xl_val_dec
    def __truediv__(self, other):
        if other == 0:
            return XlErrors.DIV_ZERO_ERROR
        return self.value / other

    @xl_val_dec
    def __rtruediv__(self, other):
        if self.value == 0:
            return XlErrors.DIV_ZERO_ERROR
        return other / self.value

    @xl_val_dec
    def __eq__(self, other: object) -> bool:
        return isinstance(other, self.__class__) and self.value == other.value

    @xl_val_dec
    def __ne__(self, other: object) -> bool:
        return not self.__eq__(other)

    def __hash__(self):
        return super().__hash__()
    
    def __str__(self):
        return str(self.value)


In [ ]:
val1 = XlValue(10)
val2 = XlValue(-10)
val3 =XlValue(XlErrors.DIV_ZERO_ERROR)

In [ ]:
s1 = pd.Series([XlValue(x) for x in  [1, 2, 3, 4, 5]])
s2 = pd.Series([XlValue(x) for x in  [1, 'uno', 3, 0, 5]])

In [ ]:
df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
df

In [ ]:
def mySum(df, axis:Literal[0, 1] | None = None):
    if axis is None:
        return df.sum(axis=0).sum()
    elif axis == 0:
        return pd.Series([df.loc[:, x].sum() for x in df.columns], index=df.columns)
    else:
        return pd.Series([df.loc[x, :].sum() for x in df.index], index=df.index)

In [ ]:
class strWrapper:
    fnc_names = [x for x in dir(pd.Series.str) if not x.startswith('_')]
    
    def __init__(self, s):
        self.s = s

    @staticmethod
    def str_dec(strfnc):
         def inner(*args, **kwargs):
            print(f'{args=}, {kwargs=}')
            answ = strfnc(*args, **kwargs)
            print(f'{answ=}')
            return answ.where(~answ.isna(), XlErrors.VALUE_ERROR)
         return inner

    def __getattr__(self, name):
           if name in self.fnc_names:
                return self.str_dec(getattr(self.s.str, name))
           raise AttributeError(f"'{name}' not found")

class MyClass:
    def __init__(self, s):
        self.s = s
    
    @property
    def str(self):
        s = pd.Series([x.value for x in self.s], index=self.s.index)
        return strWrapper(s)
    
    def __getattr__(self, name):
        return getattr(self.s, name)

In [ ]:
S2 = MyClass(s2)
S2

In [ ]:
def fixed(number, decimals=2, no_commas=False):
    import locale
    locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')

    bflag = number < 0
    number =  abs(number)
    number = round(number * 10.0**decimals, 0) * 10**(-decimals)
    n = max(0, decimals)
    answ = locale.format_string('%.{}f'.format(n), number, grouping= not no_commas)
    if bflag:
        answ =f'-{answ}'
    return pd.Series([answ])


In [1]:
import xlfunctions as xlf

In [52]:
ndate1 = xlf.date(2013, 2, 28)
answer = []
for k in range(1, 374):
    ndate2 = xlf.date(2013, 1, k)
    us_360, eu_360 = xlf.days360(ndate1, ndate2, False), xlf.days360(ndate1, ndate2, True)
    if us_360 != eu_360:
        answer.append((xlf.serial_to_date(ndate2).strftime('%d/%m/%Y'), us_360, eu_360))
answer

[('01/01/2013', -59, -57),
 ('02/01/2013', -58, -56),
 ('03/01/2013', -57, -55),
 ('04/01/2013', -56, -54),
 ('05/01/2013', -55, -53),
 ('06/01/2013', -54, -52),
 ('07/01/2013', -53, -51),
 ('08/01/2013', -52, -50),
 ('09/01/2013', -51, -49),
 ('10/01/2013', -50, -48),
 ('11/01/2013', -49, -47),
 ('12/01/2013', -48, -46),
 ('13/01/2013', -47, -45),
 ('14/01/2013', -46, -44),
 ('15/01/2013', -45, -43),
 ('16/01/2013', -44, -42),
 ('17/01/2013', -43, -41),
 ('18/01/2013', -42, -40),
 ('19/01/2013', -41, -39),
 ('20/01/2013', -40, -38),
 ('21/01/2013', -39, -37),
 ('22/01/2013', -38, -36),
 ('23/01/2013', -37, -35),
 ('24/01/2013', -36, -34),
 ('25/01/2013', -35, -33),
 ('26/01/2013', -34, -32),
 ('27/01/2013', -33, -31),
 ('28/01/2013', -32, -30),
 ('29/01/2013', -31, -29),
 ('30/01/2013', -30, -28),
 ('31/01/2013', -29, -28),
 ('01/02/2013', -29, -27),
 ('02/02/2013', -28, -26),
 ('03/02/2013', -27, -25),
 ('04/02/2013', -26, -24),
 ('05/02/2013', -25, -23),
 ('06/02/2013', -24, -22),
 

In [63]:
def xlFmlStr(tpl, with_kwargs=True):
    fnc = getattr(xlf, tpl[0])
    sig = inspect.Signature.from_callable(fnc)
    bsig = sig.bind(*tpl[1])
    if not with_kwargs or str(sig) == '(*data)':
        sig_str = ', '.join(
            f"{chr(34)}{x}{chr(34)}" if isinstance(x, str) else str(x).upper()
            for x in [*bsig.args, *bsig.kwargs.values()]
        )
    else:
        sig_str = ', '.join(
            f'{value}' if sig.parameters[key].default == inspect._empty else f'{key}={value}' 
            for key, x in bsig.arguments.items()
            if (value := f"{chr(34)}{x}{chr(34)}" if isinstance(x, str) else str(x).upper())
        )
    return f"{tpl[0].replace('_', '').upper()}({sig_str})"


In [64]:
data = [
    ('numbervalue', ['1_234,56%', ',', '_'], 12.3456),
    ('lower', ['Apt. 2B'], 'apt. 2b'),
    ('lower', ['E. E. Cummings'], 'e. e. cummings'),
    ('len', ['aBc'], 3),
    ('dollar', [1234.567, 2], '$1.234,57'),
    ('dollar', [-1234.567, -2], '($1.200)'),
    ('dollar', [-0.123, 4], '($0,1230)'),
    ('dollar', [1.5], '$1,50'),
    ('concat', ['a', 'b', 'c'], 'abc'),
    ('sum', [1, 4, 5], 10),
    ('upper', ['aBc'], 'ABC'),
    ('char', [65, 66, 67], chr(65)),
    ('code', ['abc'], ord('a')),
    ('clean', ['abc'], 'abc'),
]

In [71]:
tpl = data[0]
xlFmlStr(tpl, False)

'NUMBERVALUE("1_234,56%", ",", "_")'

In [ ]:
fmls = """=+$G$2*F13
=+F14+G14
=+SUM(F15:G15)
=+SUM(F16:G16)
=+H16+H15+H14""".split('\n')
fmls

In [ ]:
cells = '''$H$13
$H$14
$H$15
$H$16
$H$17
'''.replace('$', '').splitlines()
cells

In [ ]:
cell_to_delete = 'F'
changes = [f'{cell_to_delete}{i}' for i in range(13, 18)]
changes

In [ ]:
err_ref_cell = "'Hoja de Trabajo'!Z0"

In [ ]:
def pass_anchors(coord1, coord2):
    return coord2


In [ ]:
def shrink_range(linf, lsup, changes, del_what):
    bflag = del_what == 'col'
    k1, k2, fnc, gnc = (2, 3, ord, chr) if bflag else (3, 2, int, str)
    ucol = cell_pattern.match(lsup).group(k1)
    lx_row = cell_pattern.match(changes[0]).group(k2)
    row_changes = [
        tpl[k1 - 1] for x in changes 
        if (tpl := cell_pattern.match(x).groups()) and tpl[k2 - 1] == lx_row and tpl[k1 - 1] <= ucol
    ]
    if not row_changes:
        return linf, lsup
    min_col = fnc(row_changes[0]) if linf in changes else (fnc(row_changes[0]) - 1)
    n = len(row_changes)
    answ = [linf]
    for x in (linf, lsup)[1 - int(linf in changes):]:
        tpl = list(cell_pattern.match(x).groups())
        tpl[2 - int(bflag)] = gnc(max(min_col, (fnc(tpl[2 - int(bflag)]) - n)))
        sht, col, row = tpl
        x = f"'{sht}'!{col}{row}" if sht else f"{col}{row}"
        answ.append(x)

    return answ[-2:]


In [ ]:
del_action = 'col'
pattern = rgn_pattern
def replacement(m):
    if ':' in m[0]:
        m = tbl_pattern.match(m[0])
        prefix = f"'{m[1]}'!" if m[1] else ''
        linf, lsup = map(lambda x: f"{prefix}{x}", m[0].replace('$', '').split(':'))
        if err_ref_cell and all(x in changes for x in (linf, lsup)):
            return err_ref_cell
        linf, lsup = shrink_range(linf, lsup, changes, del_action)
        lsup = lsup.split('!')[-1]
        linf, lsup = map(lambda tpl: pass_anchors(*tpl), zip(m[0].split(':'), (linf, lsup)))
        sub_str = ':'.join([linf, lsup])
        return sub_str
    key = m[0].replace('$', '')
    return (err_ref_cell or pass_anchors(m[0], changes[key])) if key in changes else m[0]


In [ ]:
scase = ['ABC', 'EFG', 'CDEF'][0]
changes = [f'{col}{row}' for col in scase for row in range(13, 18)]
changes

In [ ]:
shrink_range('F15', 'G15', [f'G{x}' for x in range(13, 18)], 'col')

In [ ]:
def test_del_cases(scase, ncase, del_action):
        if del_action == 'col':
                changes = [f'{col}{row}' for col in scase for row in range(13, 18)]
        else:
                changes = [f'{chr(col)}{row}' for col in range(ord('A'), ord('H') + 1) for row in scase]
        fml = fmls[ncase]
        linf, lsup = rgn_pattern.search(fml)[0].split(':')
        l_answ, r_answ = shrink_range(linf, lsup, changes, del_action)
        return fml, (linf, lsup), (l_answ, r_answ)

In [ ]:
scase, ncase, del_action = range(15, 20), 0, 'row'
if del_action == 'col':
    answ = locals().get(f'del_{scase}').splitlines()[ncase]
    left, right = rgn_pattern.search(answ)[0].split(':')
else:
    fmls = '''=+SUM(A13:A17)	=+SUM(B13:B17)	=+SUM(C13:C17)	=+SUM(D13:D17)	=+SUM(E13:E17)	=+SUM(F13:F17)	=+SUM(G13:G17)'''.split('\t')
fml, tpl1, tpl2 = test_del_cases(scase, ncase, del_action)
print(fml, tpl1)
# print(tpl2, (left, right))
print(tpl2, ('left', 'right'))

In [ ]:
del_action = 'col'

for scase in ['ABC', 'EFG', 'CDEF']:
    for ncase in range(5):
        answ = locals().get(f'del_{scase}').splitlines()[ncase]
        left, right = rgn_pattern.search(answ)[0].split(':')
        fml, (linf, lsup), (l_answ, r_answ) = test_del_cases(scase, ncase, del_action)

        if not all(x == y for x, y in zip((left, right), (l_answ, r_answ))):
            print(f'*** {scase=}, {ncase=}')
            print(f'    {fml=}, {linf=}, {lsup=}, {left=}, {right=}')
            print(f'    ({l_answ=}, {l_answ == left}), ({r_answ=}, {r_answ == right})')

In [ ]:
fmls = base.split('\n')
fmls

In [ ]:
df = pd.DataFrame(fmls, columns=['fml'], index=pd.Index(cells, name='cell'))
df

In [ ]:

df.fml.str.replace(
    pattern,
    replacement,
    regex=True
)            


In [ ]:
import collections
list(collections.Counter(['B', 'C', 'B', 'C', 'A']).keys())

In [ ]:
base = '''=+SUM(A13:G13)
=+SUM(C14:G14)
=+SUM(A15:E15)
=+SUM(A16:D16)
=+SUM(A13:G17)'''


In [ ]:
del_ABC = '''=+SUM(A13:D13)
=+SUM(A14:D14)
=+SUM(A15:B15)
=+SUM(A16:A16)
=+SUM(A13:D17)'''

In [ ]:
del_EFG = '''=+SUM(A13:D13)
=+SUM(C14:D14)
=+SUM(A15:D15)
=+SUM(A16:D16)
=+SUM(A13:D17)'''

In [ ]:
del_CDEF = '''=+SUM(A13:C13)
=+SUM(C14:C14)
=+SUM(A15:B15)
=+SUM(A16:B16)
=+SUM(A13:C17)'''

In [ ]:
def xlf_code(arange):
    s = pd.Series(arange.flatten())
    return s.str[0].map(lambda x: ord(x))

In [ ]:
pd.Series(tbl['E12:E14'].values.flatten()).str[0].map(lambda x: ord(x))


In [ ]:
from xlfunctions import *

In [ ]:
def xlf_lower(*arange):
    s = pd.Series(pd.Series(arange).values.flatten())
    return s.str.lower()


In [ ]:
pd.Series([tbl['E12'].values]).values

In [ ]:
xlf_lower(tbl['E12'].values)

In [ ]:
tbl